# Chapter 10
### This is the work-along I completed going though ch 10 of Wes McKinney's book <i>Python for Data Analysis 3rd edition</i>

In [2]:
import numpy as np
import pandas as pd

## 10.1 How to Think About Group Operations

In [3]:
df = pd.DataFrame({"key1": ["a", "a", None, "b", "b", "a", None],
                  "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                  "data1": np.random.standard_normal(7),
                  "data2": np.random.standard_normal(7)})

In [4]:
df

,key1,key2,data1,data2
0,a,1,1.350546,-0.518255
1,a,2,0.056730,0.964549
2,None,1,0.733896,1.512259
3,b,2,0.105847,-0.979298
4,b,1,0.577230,0.334828
5,a,<NA>,0.917623,2.213163
6,None,1,-1.775980,-0.085017


In [5]:
grouped = df["data1"].groupby(df["key1"])

In [6]:
grouped

In [7]:
grouped.mean()

key1
a    0.774967
b    0.341539
Name: data1, dtype: float64

In [8]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()

In [9]:
means

key1  key2
a     1       1.350546
      2       0.056730
b     1       0.577230
      2       0.105847
Name: data1, dtype: float64

In [10]:
means.unstack()

key2,1,2
key1,,
a,1.350546,0.056730
b,0.577230,0.105847


In [11]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])

In [12]:
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [13]:
df["data1"].groupby([states, years]).mean()

CA  2005    0.487177
    2006    0.733896
OH  2005    0.728197
    2006   -0.599375
Name: data1, dtype: float64

In [14]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.774967,0.886485
b,1.5,0.341539,-0.322235


In [15]:
# will not work df.groupby("key2").mean()

In [16]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     1.350546 -0.518255
     2     0.056730  0.964549
b    1     0.577230  0.334828
     2     0.105847 -0.979298

In [17]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [18]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [19]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [20]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


### Iterating over Groups

In [21]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  1.350546 -0.518255
1    a     2  0.056730  0.964549
5    a  <NA>  0.917623  2.213163
b
  key1  key2     data1     data2
3    b     2  0.105847 -0.979298
4    b     1  0.577230  0.334828


In [22]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1  1.350546 -0.518255
('a', 2)
  key1  key2    data1     data2
1    a     2  0.05673  0.964549
('b', 1)
  key1  key2    data1     data2
4    b     1  0.57723  0.334828
('b', 2)
  key1  key2     data1     data2
3    b     2  0.105847 -0.979298


In [23]:
pieces = {name: group for name, group in df.groupby("key1")}

In [24]:
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.105847,-0.979298
4,b,1,0.577230,0.334828


In [25]:
grouped = df.groupby({"key1": "key", "key2": "key",
                     "data1": "data", "data2": "data"}, axis="columns")

In [26]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0  1.350546 -0.518255
1  0.056730  0.964549
2  0.733896  1.512259
3  0.105847 -0.979298
4  0.577230  0.334828
5  0.917623  2.213163
6 -1.775980 -0.085017
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [27]:
df.head()

,key1,key2,data1,data2
0,a,1,1.350546,-0.518255
1,a,2,0.056730,0.964549
2,None,1,0.733896,1.512259
3,b,2,0.105847,-0.979298
4,b,1,0.577230,0.334828


### Selecting a Column or Subset of Columns

In [28]:
df.groupby("key1")["data1"]

In [29]:
df.groupby("key1")["data2"]

In [30]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1    -0.518255
     2     0.964549
b    1     0.334828
     2    -0.979298

In [31]:
s_grouped = df.groupby(["key1", "key2"])["data2"]

In [32]:
s_grouped

In [33]:
s_grouped.mean()

key1  key2
a     1      -0.518255
      2       0.964549
b     1       0.334828
      2      -0.979298
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [34]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                     columns=["a", "b", "c", "d", "e"],
                     index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

In [35]:
people

,a,b,c,d,e
Joe,-1.354959,-1.961980,-0.486757,-0.461990,-2.081137
Steve,0.298102,0.348885,0.203251,-1.455837,1.765335
Wanda,-1.573109,-0.338919,-0.903088,0.906928,-1.008061
Jill,-0.769201,-0.969655,2.381027,1.283527,1.835390
Trey,-0.578968,1.805219,-0.789891,-0.069872,-0.463358


In [36]:
people.iloc[2:3, [1, 2]] = np.nan # add a few NA values

In [37]:
people

,a,b,c,d,e
Joe,-1.354959,-1.961980,-0.486757,-0.461990,-2.081137
Steve,0.298102,0.348885,0.203251,-1.455837,1.765335
Wanda,-1.573109,NaN,NaN,0.906928,-1.008061
Jill,-0.769201,-0.969655,2.381027,1.283527,1.835390
Trey,-0.578968,1.805219,-0.789891,-0.069872,-0.463358


In [38]:
mapping = {"a": "red", "b": "red", "c": "blue",
          "d": "blue", "e": "red", "f": "orange"}

In [39]:
by_column = people.groupby(mapping, axis="columns")

In [40]:
by_column

In [41]:
by_column.sum()

,blue,red
Joe,-0.948747,-5.398075
Steve,-1.252586,2.412323
Wanda,0.906928,-2.581170
Jill,3.664553,0.096534
Trey,-0.859763,0.762893


In [42]:
map_series = pd.Series(mapping)

In [43]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [44]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [45]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.354959,-1.961980,-0.486757,-0.461990,-2.081137
4,-1.348169,0.835564,1.591135,1.213654,1.372032
5,-1.275007,0.348885,0.203251,-0.548908,0.757273


In [46]:
key_list = ["one", "one", "one", "two", "two"]

In [47]:
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-1.354959,-1.961980,-0.486757,-0.461990,-2.081137
4,two,-0.769201,-0.969655,-0.789891,-0.069872,-0.463358
5,one,-1.573109,0.348885,0.203251,-1.455837,-1.008061


### Grouping by Index Levels

In [48]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                   names=["city", "tenor"])

In [49]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)

In [50]:
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.422827  0.533752  1.375677  0.329622  2.698300
1     -0.358169 -0.104828 -0.005578 -0.202483  0.146069
2      0.678933 -0.394049  1.178673 -0.116473 -1.032566
3     -0.121115  0.415355  0.711930  1.814383 -0.723973

In [51]:
hier_df.groupby(level="city", axis="columns").count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 Data Aggregation

In [52]:
df

,key1,key2,data1,data2
0,a,1,1.350546,-0.518255
1,a,2,0.056730,0.964549
2,None,1,0.733896,1.512259
3,b,2,0.105847,-0.979298
4,b,1,0.577230,0.334828
5,a,<NA>,0.917623,2.213163
6,None,1,-1.775980,-0.085017


In [53]:
grouped = df.groupby("key1")

In [54]:
grouped

In [55]:
grouped["data1"].nsmallest(2)

key1   
a     1    0.056730
      5    0.917623
b     3    0.105847
      4    0.577230
Name: data1, dtype: float64

In [56]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [57]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.293816,2.731418
b,1,0.471383,1.314126


In [58]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.774967  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.341539  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     1.134085  1.350546   3.0  0.886485  1.367381 -0.518255  0.223147   
b     0.459384  0.577230   2.0 -0.322235  0.929227 -0.979298 -0.650767   

                                    
           50%       75%       max  
key1                                
a     0.964549  1.588856  2.213163  
b    -0.322235  0.006296  0.334828  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [59]:
tips = pd.read_csv("/kaggle/input/tipsdata/tips.csv")

In [60]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [61]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]

In [62]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [63]:
grouped = tips.groupby(["day", "smoker"])

In [64]:
grouped_pct = grouped["tip_pct"]

In [65]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [66]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [67]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [68]:
functions = ["count", "mean", "max"]

In [69]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)

In [70]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [71]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [72]:
ftuples = [("Average", "mean"), ("Variance", np.var)]

In [73]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [74]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [75]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"], "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [76]:
ex1 = tips.groupby(["day", "smoker"], as_index=False)

In [77]:
ex1.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 Apply: General split-apply-combine

In [78]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [79]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [80]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [81]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [82]:
tips["total_bill"].max()

50.81

In [83]:
result = tips.groupby("smoker")["tip_pct"].describe()

In [84]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [85]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [86]:
def f(group):
    return group.describe()

In [87]:
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suppressing the Group Keys

In [88]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Quantile and Bucket Analysis

In [89]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                     "data2": np.random.standard_normal(1000)})

In [90]:
frame.head()

,data1,data2
0,-0.549846,-1.685091
1,-1.895337,-0.935738
2,-0.744929,-0.430582
3,1.969569,-0.576024
4,0.842001,0.427532


In [91]:
quartiles = pd.cut(frame["data1"], 4)

In [92]:
quartiles.head(10)

0    (-1.487, -0.0165]
1     (-2.963, -1.487]
2    (-1.487, -0.0165]
3       (1.454, 2.924]
4     (-0.0165, 1.454]
5     (-0.0165, 1.454]
6    (-1.487, -0.0165]
7     (-0.0165, 1.454]
8    (-1.487, -0.0165]
9     (-0.0165, 1.454]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.963, -1.487] < (-1.487, -0.0165] < (-0.0165, 1.454] < (1.454, 2.924]]

In [93]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

In [94]:
grouped = frame.groupby(quartiles)

In [95]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                       
(-2.963, -1.487]  data1 -2.957441 -1.510556     79 -1.864933
                  data2 -2.557832  2.527231     79  0.020791
(-1.487, -0.0165] data1 -1.481894 -0.018599    416 -0.634148
                  data2 -2.866315  3.729974    416 -0.060058
(-0.0165, 1.454]  data1 -0.011354  1.448328    432  0.580633
                  data2 -2.389139  2.807091    432  0.012835
(1.454, 2.924]    data1  1.459145  2.924470     73  1.927160
                  data2 -1.782785  2.012667     73  0.179571

In [96]:
# same thing as above
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                        min       max count      mean       min       max   
data1                                                                       
(-2.963, -1.487]  -2.957441 -1.510556    79 -1.864933 -2.557832  2.527231   
(-1.487, -0.0165] -1.481894 -0.018599   416 -0.634148 -2.866315  3.729974   
(-0.0165, 1.454]  -0.011354  1.448328   432  0.580633 -2.389139  2.807091   
(1.454, 2.924]     1.459145  2.924470    73  1.927160 -1.782785  2.012667   

                                   
                  count      mean  
data1                              
(-2.963, -1.487]     79  0.020791  
(-1.487, -0.0165]   416 -0.060058  
(-0.0165, 1.454]    432  0.012835  
(1.454, 2.924]       73  0.179571

In [97]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)

In [98]:
quartiles_samp.head()

0    1
1    0
2    0
3    3
4    3
Name: data1, dtype: int64

In [99]:
grouped = frame.groupby(quartiles_samp)

In [100]:
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.957441 -0.730323    250 -1.300005
      data2 -2.866315  2.767376    250 -0.002688
1     data1 -0.730232 -0.006974    250 -0.344716
      data2 -2.819175  3.729974    250 -0.102876
2     data1 -0.006731  0.623907    250  0.309161
      data2 -2.256525  2.807091    250  0.077006
3     data1  0.626644  2.924470    250  1.257083
      data2 -2.389139  2.199476    250  0.009807

### Example: Filling Missing Values with Group-Specific Values

In [101]:
s = pd.Series(np.random.standard_normal(6))

In [102]:
s[::2] = np.nan

In [103]:
s

0         NaN
1    0.378760
2         NaN
3   -1.178234
4         NaN
5    1.505276
dtype: float64

In [104]:
s.fillna(s.mean())

0    0.235267
1    0.378760
2    0.235267
3   -1.178234
4    0.235267
5    1.505276
dtype: float64

In [105]:
states = ["Ohio", "New York", "Vermont", "Florida", "Oregon", "Nevada", "California", "Idaho"]

In [106]:
group_key = ["East", "East", "East", "East", "West", "West", "West", "West"]

In [107]:
data = pd.Series(np.random.standard_normal(8), index=states)

In [108]:
data

Ohio          1.296608
New York     -1.448622
Vermont       1.640045
Florida      -0.096091
Oregon        1.985375
Nevada       -1.029804
California    0.329719
Idaho         1.634448
dtype: float64

In [109]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan

In [110]:
data

Ohio          1.296608
New York     -1.448622
Vermont            NaN
Florida      -0.096091
Oregon        1.985375
Nevada             NaN
California    0.329719
Idaho              NaN
dtype: float64

In [111]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [112]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [113]:
data.groupby(group_key).mean()

East   -0.082702
West    1.157547
dtype: float64

In [114]:
def fill_mean(group):
    return group.fillna(group.mean())

In [115]:
data.groupby(group_key).apply(fill_mean)

East  Ohio          1.296608
      New York     -1.448622
      Vermont      -0.082702
      Florida      -0.096091
West  Oregon        1.985375
      Nevada        1.157547
      California    0.329719
      Idaho         1.157547
dtype: float64

In [116]:
fill_values = {"East": 0.5, "West": -1}

In [117]:
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [118]:
data.groupby(group_key).apply(fill_func)

East  Ohio          1.296608
      New York     -1.448622
      Vermont       0.500000
      Florida      -0.096091
West  Oregon        1.985375
      Nevada       -1.000000
      California    0.329719
      Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [119]:
suits = ["H", "S", "C", "D"] # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [120]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [121]:
def draw(deck, n=5):
    return deck.sample(n)

In [122]:
draw(deck)

10C    10
3S      3
10S    10
KS     10
2H      2
dtype: int64

In [123]:
def get_suit(card):
    # last letter is suit
    return card[-1]

In [124]:
deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   5C      5
D  4D      4
   10D    10
H  6H      6
   7H      7
S  8S      8
   3S      3
dtype: int64

In [125]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

10C    10
6C      6
KD     10
AD      1
8H      8
5H      5
KS     10
5S      5
dtype: int64

## Example: Group Weighted Average and Correlation

In [126]:
df = pd.DataFrame({"category": ["a", "a", "a", "a", "b", "b", "b", "b"],
                  "data": np.random.standard_normal(8),
                  "weights": np.random.uniform(size=8)})

In [127]:
df

,category,data,weights
0,a,0.749208,0.430800
1,a,0.197007,0.626501
2,a,-1.166499,0.749379
3,a,-0.887634,0.742447
4,b,-1.263691,0.475563
5,b,-0.914810,0.454154
6,b,0.925397,0.159357
7,b,-0.622142,0.114652


In [128]:
grouped = df.groupby("category")

In [129]:
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

In [130]:
grouped.apply(get_wavg)

category
a   -0.426416
b   -0.781150
dtype: float64

In [131]:
close_px = pd.read_csv("/kaggle/input/stockexamp/stock_px.csv", parse_dates=True, index_col=0)

In [132]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [133]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [134]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [135]:
rets = close_px.pct_change().dropna()

In [136]:
def get_year(x):
    return x.year

In [137]:
by_year = rets.groupby(get_year)

In [138]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [139]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])

In [140]:
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Esample: Group-Wise Linear Regression

In [141]:
import statsmodels.api as sm

def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1. 
    result = sm.OLS(Y, X).fit()
    return result.params

In [142]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 Group Transforms and "Unwrapped" GroupBys

In [143]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                  'value': np.arange(12.)})

In [144]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [145]:
g = df.groupby('key')['value']

In [146]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [147]:
def get_mean(group):
    return group.mean()

In [148]:
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [149]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [150]:
def times_two(group):
    return group * 2

In [151]:
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [152]:
def get_ranks(group):
    return group.rank(ascending=False)

In [153]:
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [154]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [155]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [156]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [157]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [158]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [159]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# 10.5 Pivot Tables and Cross-Tabulation

In [160]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [163]:
tips.pivot_table(index=["day", "smoker"],
                values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [164]:
tips.pivot_table(index=["time", "day"], columns="smoker", values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [165]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [166]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [167]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## Cross-Tabulations: Crosstab

In [168]:
from io import StringIO

In [169]:
data = """Sample Nationality Handedness
1    USA    Right-handed
2    Japan    Left-handed
3    USA    Right-handed
4    Japan    Right-handed
5    Japan    Left-handed
6    Japan    Right-handed
7    USA    Right-handed
8    USA    Left-handed
9    Japan    Right-handed
10    USA    Right-handed"""

In [170]:
data = pd.read_table(StringIO(data), sep="\s+")

In [171]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [172]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [173]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244